In [16]:
import pandas as pd

In [17]:
df = pd.read_csv('tweets_remaining_09042020_16072020.csv', sep=';')
df.head()

,id,created_at,full_text
0,1,2020-04-09 23:59:51+00:00,@KennyDegu very very little volume. With $10T ...
1,2,2020-04-09 23:58:55+00:00,#ES_F achieved Target 2780 closing above 50% #...
2,3,2020-04-09 23:58:52+00:00,RT @KimbleCharting: Silver/Gold indicator crea...
3,4,2020-04-09 23:58:27+00:00,@Issaquahfunds Hedged our $MSFT position into ...
4,5,2020-04-09 23:57:59+00:00,RT @zipillinois: 3 Surprisingly Controversial ...


In [18]:
df.isna().sum()

id            0
created_at    0
full_text     0
dtype: int64

In [19]:
# change the 'full_text' column to 'text'
df.rename(columns={'full_text': 'text'}, inplace=True)

In [20]:
# extract all tickets in the format $TICKER from the 'text' column
# the ticker must consist of letters only (no digits)
df['ticker'] = df['text'].str.findall(r'\$[A-Za-z]+')

# remove the dollar sign and convert tickers to uppercase
df['ticker'] = df['ticker'].apply(lambda x: [t.replace('$', '').upper() for t in x])

# manually explode the list into individual rows by expanding the DataFrame
df_expanded = df.explode('ticker', ignore_index=True)

df_expanded


,id,created_at,text,ticker
0,1,2020-04-09 23:59:51+00:00,@KennyDegu very very little volume. With $10T ...,SPX
1,2,2020-04-09 23:58:55+00:00,#ES_F achieved Target 2780 closing above 50% #...,SPX
2,2,2020-04-09 23:58:55+00:00,#ES_F achieved Target 2780 closing above 50% #...,SPY
3,3,2020-04-09 23:58:52+00:00,RT @KimbleCharting: Silver/Gold indicator crea...,NaN
4,4,2020-04-09 23:58:27+00:00,@Issaquahfunds Hedged our $MSFT position into ...,MSFT
...,...,...,...,...
2767284,938672,2020-07-16 00:00:00+00:00,$AAL $TWTR $SPCE $AZN $ERI $RCL $JNJ $AA $SNBR...,RCL
2767285,938672,2020-07-16 00:00:00+00:00,$AAL $TWTR $SPCE $AZN $ERI $RCL $JNJ $AA $SNBR...,JNJ
2767286,938672,2020-07-16 00:00:00+00:00,$AAL $TWTR $SPCE $AZN $ERI $RCL $JNJ $AA $SNBR...,AA
2767287,938672,2020-07-16 00:00:00+00:00,$AAL $TWTR $SPCE $AZN $ERI $RCL $JNJ $AA $SNBR...,SNBR


In [21]:
df = df_expanded

In [22]:
df

,id,created_at,text,ticker
0,1,2020-04-09 23:59:51+00:00,@KennyDegu very very little volume. With $10T ...,SPX
1,2,2020-04-09 23:58:55+00:00,#ES_F achieved Target 2780 closing above 50% #...,SPX
2,2,2020-04-09 23:58:55+00:00,#ES_F achieved Target 2780 closing above 50% #...,SPY
3,3,2020-04-09 23:58:52+00:00,RT @KimbleCharting: Silver/Gold indicator crea...,NaN
4,4,2020-04-09 23:58:27+00:00,@Issaquahfunds Hedged our $MSFT position into ...,MSFT
...,...,...,...,...
2767284,938672,2020-07-16 00:00:00+00:00,$AAL $TWTR $SPCE $AZN $ERI $RCL $JNJ $AA $SNBR...,RCL
2767285,938672,2020-07-16 00:00:00+00:00,$AAL $TWTR $SPCE $AZN $ERI $RCL $JNJ $AA $SNBR...,JNJ
2767286,938672,2020-07-16 00:00:00+00:00,$AAL $TWTR $SPCE $AZN $ERI $RCL $JNJ $AA $SNBR...,AA
2767287,938672,2020-07-16 00:00:00+00:00,$AAL $TWTR $SPCE $AZN $ERI $RCL $JNJ $AA $SNBR...,SNBR


In [23]:
df['ticker'].value_counts()

ticker
SPX       165924
AAPL      119745
SPY       111267
AMZN      106243
FB         83612
           ...  
DJUSCT         1
ALIG           1
IONC           1
IONKF          1
AIRW           1
Name: count, Length: 15278, dtype: int64

In [24]:
import yfinance as yf

def is_valid_ticker(ticker):
    # load data for a given ticker
    stock = yf.Ticker(ticker)
    try:
        # try to get info
        info = stock.info
        return info is not None
    except:
        # raise an exception if it fails
        return False

In [25]:
import os

possible_tickers  = df['ticker'].unique()

# convert all tickers to string
possible_tickers = [str(ticker) for ticker in possible_tickers]

# ignore the numbers
possible_tickers = [ticker for ticker in possible_tickers if not ticker.isdigit()]


# commented out because it executes for too long
# I saved the valid tickers into txt file

output = 'valid_tickers_2.txt'

#execute if there is no output file in the directory
if not os.path.exists(output):
    with open(output, 'w') as file:
        for ticker in possible_tickers:
            if is_valid_ticker(ticker):
                file.write(ticker + '\n')


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/APRN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=APRN&crumb=YFPP%2FUxFGKG
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/NOVN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=NOVN&crumb=YFPP%2FUxFGKG
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/EURAUD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=EURAUD&crumb=YFPP%2FUxFGKG
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CHFJPY?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=

In [26]:
path = 'valid_tickers_2.txt'

valid_tickers = []

with open(path, 'r') as file:
    for line in file:
        clean_line = line.strip().replace('[', '').replace(']', '').replace("'", "").replace(',', '')
        
        if clean_line:
            valid_tickers.append(clean_line)

print(valid_tickers)


['SPX', 'SPY', 'nan', 'MSFT', 'GILD', 'MO', 'PFE', 'JNK', 'USD', 'AAPL', 'UMRX', 'DECN', 'OPGN', 'CODX', 'HTBX', 'TNXP', 'BA', 'APRN', 'JNUG', 'PCTL', 'BIEI', 'NBY', 'UBER', 'WTRH', 'NOVN', 'TSLA', 'XSPA', 'INO', 'ASTC', 'MRNA', 'GPMT', 'SPXS', 'QQQ', 'NUGT', 'BURG', 'VXRT', 'TVIX', 'IBIO', 'ACB', 'TLRY', 'CLIR', 'BKYI', 'TOPS', 'NVAX', 'GME', 'TTOO', 'AYTU', 'FXY', 'FXA', 'OIL', 'FXB', 'ACWI', 'FXE', 'FB', 'NDX', 'BIIB', 'BNTX', 'AMZN', 'COST', 'WMT', 'PGR', 'NRG', 'DS', 'BPFH', 'FAST', 'A', 'CTL', 'CVBF', 'CYOU', 'GES', 'TTI', 'JNJ', 'RO', 'TGT', 'DG', 'MCD', 'EMB', 'CMCSA', 'EURAUD', 'CHFJPY', 'EURUSD', 'EURGBP', 'AUDJPY', 'XAUUSD', 'EURCHF', 'VXX', 'ABT', 'TMO', 'MDT', 'LNG', 'V', 'ZM', 'GOOGL', 'SLV', 'JPM', 'GCI', 'ICD', 'CLMT', 'ACY', 'TLSA', 'NLS', 'BABA', 'GBTC', 'DIS', 'SSO', 'UPRO', 'PTLC', 'QLD', 'DOW', 'DDM', 'VIX', 'DAX', 'CAC', 'FTSE', 'XOM', 'CVX', 'ES', 'BIOL', 'BIOC', 'CCL', 'I', 'WHR', 'CAT', 'IRT', 'PDN', 'BOE', 'BMN', 'BAC', 'FED', 'USO', 'GLD', 'ROKU', 'LVGO', 'AY

In [27]:
# having the valid tickers, we can now filter the dataframe to only include the valid tickers
df = df[df['ticker'].isin(valid_tickers)]
df

,id,created_at,text,ticker
0,1,2020-04-09 23:59:51+00:00,@KennyDegu very very little volume. With $10T ...,SPX
1,2,2020-04-09 23:58:55+00:00,#ES_F achieved Target 2780 closing above 50% #...,SPX
2,2,2020-04-09 23:58:55+00:00,#ES_F achieved Target 2780 closing above 50% #...,SPY
4,4,2020-04-09 23:58:27+00:00,@Issaquahfunds Hedged our $MSFT position into ...,MSFT
5,5,2020-04-09 23:57:59+00:00,RT @zipillinois: 3 Surprisingly Controversial ...,GILD
...,...,...,...,...
2767284,938672,2020-07-16 00:00:00+00:00,$AAL $TWTR $SPCE $AZN $ERI $RCL $JNJ $AA $SNBR...,RCL
2767285,938672,2020-07-16 00:00:00+00:00,$AAL $TWTR $SPCE $AZN $ERI $RCL $JNJ $AA $SNBR...,JNJ
2767286,938672,2020-07-16 00:00:00+00:00,$AAL $TWTR $SPCE $AZN $ERI $RCL $JNJ $AA $SNBR...,AA
2767287,938672,2020-07-16 00:00:00+00:00,$AAL $TWTR $SPCE $AZN $ERI $RCL $JNJ $AA $SNBR...,SNBR


In [29]:
# after a brief analysis I noticed, that there are some problems with the 'text' column

#the problem is that some text contain ; symbol, so we need to take care of that
df['text'] = df['text'].str.replace(';', ',')
#we also need to clear all new line signs from the text
df['text'] = df['text'].str.replace('\n', ' ')

# I replace the semicolon with a coma and new line with a space, what is not ideal, but will work

C:\Users\48726\AppData\Local\Temp\ipykernel_13672\3701072222.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.replace(';', ',')
C:\Users\48726\AppData\Local\Temp\ipykernel_13672\3701072222.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.replace('\n', ' ')


In [30]:
df.to_csv('tweets_remaining_cleared.csv', sep=';', index=False)